In [1]:
# Default locations if parameter not passed
benchmark_data_path = "tb-compare-coaraci.pkl"

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from rich import print as rprint
import seaborn as sns
import os
import re
from scipy.stats import bootstrap

benchmark_data = pickle.load(open(benchmark_data_path, "rb"))

df = pd.DataFrame(benchmark_data["dataframe"])

sns.set_theme()

ModuleNotFoundError: No module named 'pandas'

In [ ]:
bench_names = df["name"].unique()
print(f"Benchmarks found: {bench_names}")
run_metadata = benchmark_data["config"].applications
n_runs = benchmark_data["config"].metadata.runs

df_bar = df[df["name"] == 'tb-compare']


n_runs = benchmark_data["config"].metadata.runs
print(f"Number of runs: {n_runs}")
print(df_bar.columns)
print(df_bar.head())

In [ ]:
dump_to_csv = False

if dump_to_csv:
    csv_path = os.path.join(os.path.dirname(benchmark_data_path), "tb-compare.csv")
    print(f"Dumping to {csv_path}")
    df_bar.to_csv(csv_path, index=False)

In [ ]:
# tb_time field (seconds)
# get total time from the output
def extract_elapsed_time(text):
    matches = re.findall(r"Elapsed Time ([\d.eE+-]+) seconds", text)
    if matches:
        # gets last mach
        # float() already handles scientific notation
        return float(matches[-1])
    return float('nan')

def extract_total_sched_time(text):
    # Find all "Scheduling : <number>"
    matches = re.findall(r"Scheduling\s*:\s*(\d+)", text)
    if not matches:
        print("No scheduling time found in the text:")
        print(text)
        return None
    total_microseconds = sum(int(m) for m in matches)
    return total_microseconds / 1e6  # microseconds to seconds

def map_schedueler_name(image_name):
    if "improv" in image_name:
        return "Record Replay"
    return "Baseline"

def map_scheduler(row):
    if 'improv' in row['image'].lower():
        return 'Record Replay'
    if row['scheduler'] == 'roundrobin':
        return 'Baseline-RR'
    if row['scheduler'] == 'heft':
        return 'Baseline-HEFT'
    return np.nan  # or 'Baseline-Unknown'

df_bar['tb_time'] = df_bar['full_output'].apply(extract_elapsed_time)
df_bar['sched_time'] = df_bar['full_output'].apply(extract_total_sched_time)
# worker nodes is node_count - 1
df_bar['worker_nodes'] = df_bar['node_count'].apply(lambda x: int(x - 1) if x > 1 else None)
# df_bench['scheduler_name'] = df_bench['image'].apply(map_schedueler_name)
df_bar['scheduler_name'] = df_bar.apply(map_scheduler, axis=1)

print(df_bar.columns)



# Experiment 1

freeze, for now:
kernel: compute_bound
type: fft

freeze:
ompc_tb_iter_num: 100
ompc_init_iter: 5
ompc_resch_iter: 0

y axsis: tb_time
x axis: worker node count 

one line per iter

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

for iter in df_bar['iter'].unique():
  print(f"Processing iteration: {iter}")
  for type in df_bar['type'].unique():
    print(f"Processing type: {type}")
    ## TODO: keep doing this for kernel and ompc_tb_iter_num

    # apply filter for this loop
    df_plot = df_bar[(df_bar['iter'] == iter) & (df_bar['type'] == type)].copy()
    df_plot = df_plot[['tb_time', 'worker_nodes', 'scheduler_name']].copy()
    df_plot = df_plot.dropna() # worker nodes can be None if np = 1

    # 2) draw the plot
    plt.figure()
    sns.lineplot(
        data=df_plot,
        x='worker_nodes',
        y='tb_time',
        hue='scheduler_name',
        style='scheduler_name',
        markers=True,
        dashes=False,
        errorbar='ci',
    )

    plt.xlabel('Worker Nodes')
    plt.ylabel('Elapsed time (s)')
    plt.title(f'{type.upper()} Dependency, {iter} iters. Elapsed time per worker node.')
    plt.xticks(sorted(df_plot['worker_nodes'].unique()))
    plt.legend(title='Scheduler')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Experiment 2

Show two stacked bars per worker node count.
First stacked bar is: 'Record Replay' scheduler total scheduling stacked over interpolation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

def extract_interpolation_time(text):
  # matchs all interpolation time and return sum of float
  matches = re.findall(r"Interpolation : ([\d.eE+-]+)", text)

  if matches:
    # Convert all matches to float and sum them up
    total_time = sum(float(match) for match in matches)
    return total_time / 1e6  # Convert microseconds to seconds
  return float('nan')

# 1) Prepare the data: only Record Replay has interpolation
df_bar = df_bar.copy()
df_bar['interp_time'] = 0.0
mask_rr = df_bar['scheduler_name'] == 'Record Replay'
df_bar.loc[mask_rr, 'interp_time'] = df_bar['full_output'].apply(extract_interpolation_time)

df_bar = df_bar[['worker_nodes', 'scheduler_name', 'sched_time', 'interp_time']].copy()
# drop if NaN in worker_nodes
df_bar = df_bar.dropna(subset=['worker_nodes'])


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

scheduler_order = ['Record Replay', 'Baseline-HEFT', 'Baseline-RR']

# 6 colors: [RR_sched, RR_interp, HEFT_sched, HEFT_interp, RRr_sched, RRr_interp]
colors = sns.color_palette("colorblind", len(scheduler_order) * 2)

# bar width so total group width ≈ 0.8
n_sched = len(scheduler_order)
width = 0.8 / n_sched

# filter original df_bench, not df_bar
df_loop = df_bar.copy()

# 1) Aggregate means
agg = (
    df_loop
    .groupby(['worker_nodes', 'scheduler_name'])[['sched_time', 'interp_time']]
    .mean()
    .reset_index()
)

# 2) Pivot for stacking
sched = (
    agg
    .pivot(index='worker_nodes', columns='scheduler_name', values='sched_time')
    [scheduler_order]
)
interp = (
    agg
    .pivot(index='worker_nodes', columns='scheduler_name', values='interp_time')
    .fillna(0)[scheduler_order]
)

# 3) Plot
nodes = sched.index.values
x = np.arange(len(nodes))

fig, ax = plt.subplots(figsize=(8, 5))
for i, name in enumerate(scheduler_order):
    # center the three bars at each x
    offset = (i - (n_sched - 1) / 2) * width
    pos = x + offset

    # scheduling segment
    ax.bar(
        pos,
        sched[name],
        width,
        label=f'{name} – Scheduling',
        color=colors[2*i]
    )
    # interpolation segment on top
    ax.bar(
        pos,
        interp[name],
        width,
        bottom=sched[name],
        label=f'{name} – Interpolation',
        color=colors[2*i + 1]
    )

# formatting
ax.set_xticks(x)
ax.set_xticklabels(nodes)
ax.set_xlabel("Worker Nodes")
ax.set_ylabel("Time (s)")
ax.set_title(f"FFT Dependency – Iter = {iter_val}, Type = {type_val}")
ax.legend()
ax.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()
